In [ ]:
# Copyright 2020, The TensorFlow Federated Authors.
# Copyright 2020, Ronald Seoh
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

import simple_fedavg_tf
import simple_fedavg_tff

## Experiment Settings

In [ ]:
total_rounds = 256 # Number of total training rounds
rounds_per_eval = 1 # How often to evaluate
train_clients_per_round = 2 # How many clients to sample per round.
client_epochs_per_round = 1 # Number of epochs in the client to take per round.
batch_size = 20 # Batch size used on the client.
test_batch_size = 100 # Minibatch size of test data.

# Optimizer configuration
server_learning_rate = 1.0 # Server learning rate.
fclient_learning_rate = 0.1 # Client learning rate

## Training

In [ ]:
train_data, test_data = get_shakespeare_dataset()

def tff_model_fn():
    """Constructs a fully initialized model for use in federated averaging."""
    keras_model = create_original_fedavg_mobilebert_model()
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    return simple_fedavg_tf.KerasModelWrapper(keras_model,
                                              test_data.element_spec, loss)

iterative_process = simple_fedavg_tff.build_federated_averaging_process(
    tff_model_fn, server_optimizer_fn, client_optimizer_fn)
server_state = iterative_process.initialize()

metric = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
model = tff_model_fn()

for round_num in range(total_rounds):
    sampled_clients = np.random.choice(
        train_data.client_ids,
        size=train_clients_per_round,
        replace=False)
    sampled_train_data = [
        train_data.create_tf_dataset_for_client(client)
        for client in sampled_clients
    ]
    server_state, train_metrics = iterative_process.next(
        server_state, sampled_train_data)
    print(f'Round {round_num} training loss: {train_metrics}')

    if round_num % rounds_per_eval == 0:
        model.from_weights(server_state.model_weights)
        accuracy = simple_fedavg_tf.keras_evaluate(model.keras_model, test_data,
                                                 metric)
        print(f'Round {round_num} validation accuracy: {accuracy * 100.0}')